# Imports

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.compat.v1.saved_model import load, tag_constants

# Constants

In [2]:
X_TEST_PATH = 'load_data/load_xtest.csv'
Y_TEST_PATH = 'load_data/load_ytest.csv'

# Load the Data

In [3]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)
print(f'Shape of y_test {y_test.shape}')

Shape of y_test (10000,)


In [4]:
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=float)
print(f'Shape of x_test {x_test.shape}')

Shape of x_test (10000, 784)


# Create Session

In [5]:
# เวลาเราจะทำอะไรเกี่ยวกับ tensorflow อะ ใช้ 2 อย่าง graph, session
graph = tf.Graph()
session = tf.Session(graph=graph)

# Load the Model

```
tf.compat.v1.saved_model.load(sess,
                           [tag_constants.SERVING],
                           EXPORT_DIR)
```

In [11]:
load(sess=session, tags=[tag_constants.SERVING], export_dir='SavedModel')
# เรื่อง tags นั้น tag ของเราได้โดนกำหนดค่าตั้งแต่ตอนที่เราทำการ save model ก่อนหน้านี้แล้ว แล้ว default ก็เป็นตัวนี้

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from SavedModel\variables\variables


meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "ApplyAdam"
      input_arg {
        name: "var"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "m"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "v"
        type_attr: "T

# Run Session & Make Prediction

In [22]:
X = graph.get_tensor_by_name('X:0')  # get the tensor from the graph with the specific name
# the reason that follow with :0 is to get the values from the operation
# arrow in tensorflow graph are operations, if you fill it with 'X', you till access to the operations itself
# so when you want to access the value or end point of operation just put ':0' at the end of the name
# the reason it is 0 is, there might be more than 1 output from that operation
# if that's is, the ':0' will be index for the first one, later :1 :2 :3 ...
y_prediction = graph.get_tensor_by_name('accuracy_calc/prediction:0')  
# accuracy_calc/prediction is operation ArgMax
# then we access to its values by specify ':0'

# Check Results

In [23]:
# everytime we actually want to use tensorflow to make a calculation or give us an output of some sort
# we have to run session
prediction = session.run(feed_dict={X:x_test}, fetches=y_prediction)

In [17]:
prediction

array([7, 2, 1, ..., 4, 0, 6], dtype=int64)

In [18]:
y_test

array([7, 2, 1, ..., 4, 5, 6])

In [28]:
number_correct = np.equal(prediction, y_test).sum()
print('Accuracy : {:.02}'.format(number_correct/y_test.shape[0]))

Accuracy : 0.89


# Cleanup

In [29]:
session.close()
tf.reset_default_graph()